In [49]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torchvision
from torchvision import datasets,transforms
from tqdm import tqdm
import cv2
from torch.utils.data import Dataset,DataLoader
import torch.optim as optim
from PIL import Image
import torchvision.transforms as transforms
import os
import torch.nn.functional as F
from modelLib.models.PoseDetector import PoseDetector

In [10]:
classified_path = '../datasets/classifiedData'
dirs = ['Bhuj','Padam','Shav','Tada','Trik','Vriksh']
for x in dirs:
    if x not in os.listdir(classified_path):
        path = os.path.join(classified_path,x)
        os.mkdir(path)

In [11]:
videos_path = '../datasets/YogaVidCollected/Yoga_Vid_Collected'
videos = os.listdir(videos_path)

In [58]:
def get_frames_keypoints(filename, n_frames=1):
    frames = []
    PD = PoseDetector(True, 2, False, 0.5)
    v_cap = cv2.VideoCapture(filename)
    v_len = int(v_cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(v_len)
    frame_list = np.linspace(0, v_len-1, n_frames+1, dtype=np.int16)
    print(frame_list)
    for fn in range(v_len):
        success, frame = v_cap.read()
        if success is False:
            continue
        if (fn in frame_list):
            frame = np.array(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            keypoints = PD.detectJoints(frame)
            frame = np.transpose(frame,(2,0,1))
            frames.append({"frame" : frame,"pose_keypoints":keypoints.pose_landmarks})
    v_cap.release()
    return frames, v_len

In [59]:
class PoseDataset(Dataset):
    def __init__(self, videos_dir, transform=None):
        self.videos_dir = videos_dir
        self.transform = transform
        self.videos = os.listdir(self.videos_dir)

    def __len__(self):
        return len(self.videos)

    def __getitem__(self, index):
        video_path = os.path.join(self.videos_dir, self.videos[index])
        # frames,v_len = utils.
        frames, v_len = get_frames_keypoints(video_path, n_frames=45)
        label = self.videos[index][self.videos[index].find(
            "_")+1:self.videos[index].find(".")]
        if self.transform is not None:
            pass

        return {
            "frame_keypoints": frames,
            "label": label,
            # "v_len":v_len
            }

In [62]:
# example demonstrating the use of get_frames function
frames,v_len = get_frames_keypoints(videos_path+'/Abhay_Bhuj.mp4',n_frames=45)

545
[  0  12  24  36  48  60  72  84  96 108 120 132 145 157 169 181 193 205
 217 229 241 253 265 278 290 302 314 326 338 350 362 374 386 398 411 423
 435 447 459 471 483 495 507 519 531 544]


In [64]:
frames[0]

{'frame': array([[[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],
 
        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [1, 1, 1, ..., 3, 3, 3],
         [1, 1, 1, ..., 3, 3, 3],
         [1, 1, 1, ..., 3, 3, 3]],
 
        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [6, 6, 6, ..., 7, 7, 7],
         [6, 6, 6, ..., 7, 7, 7],
         [6, 6, 6, ..., 7, 7, 7]]], dtype=uint8),
 'pose_keypoints': landmark {
   x: 0.6347312331199646
   y: 0.7926359176635742
   z: -0.08882799744606018
   visibility: 0.9999979734420776
 }
 landmark {
   x: 0.6403542757034302
   y: 0.7834061980247498
   z: -0.080195851624012
   visibility: 0.999996542930603
 }
 landmark {
   x: 0.6406067609786987
   y: 0.783

'Bhuj'